In [1]:
!pip install mysql-connector-python

In [ ]:
import tkinter as tk
from tkinter import messagebox
from tkinter.ttk import Combobox
import mysql.connector

con = mysql.connector.connect(
    host="localhost",
    user="root",
    password="mysql",
    database="airline_reservation"
)

def return_to_home(root_window):
    root_window.destroy()
    rootp.deiconify()

def cancel_reservation():
    rootp.withdraw() 
    root2 = tk.Toplevel()
    root2.title("Welcome, Customer To our Cancellation System")
    root2.geometry("800x600")

    tk.Label(root2, text="Enter last 4 digits of your Citizenship Number").pack()
    e1 = tk.Entry(root2)
    e1.pack()

    tk.Label(root2, text="Choose your class").pack()
    w2 = Combobox(root2, height=15, width=15, values=["BusinessClass", "Economic"])
    w2.pack()

    tk.Label(root2, text="Select boarding").pack()
    w3 = Combobox(root2, height=15, width=15, values=["New York", "Chicago", "Dallas", "San Francisco"])
    w3.pack()

    def cancel():
        adno = e1.get()
        b = w2.get()
        c = w3.get()
        
        cur = con.cursor()
        if adno == '' or b == '' or c == '':
            messagebox.showerror("Oops", "You can't leave any field empty")
        else:
            if b.lower() == "economic":
                cur.execute("SELECT * FROM economic WHERE adno=%s AND boarding=%s", (adno, c))
            else:
                cur.execute("SELECT * FROM business WHERE adno=%s AND boarding=%s", (adno, c))
                
            records = cur.fetchall()
            if not records:
                messagebox.showinfo("No Seats Found", "No seats found for cancellation.")
            else:
                if b.lower() == "economic":
                    cur.execute("DELETE FROM economic WHERE adno=%s AND boarding=%s", (adno, c))
                else:
                    cur.execute("DELETE FROM business WHERE adno=%s AND boarding=%s", (adno, c))
                    
                con.commit()
                messagebox.showinfo("Reservation Cancelled", "Your reservation has been cancelled.")
                
        return_to_home(root2)  
    
    tk.Button(root2, text="Cancel Reservation", command=cancel, bg="red", fg="white").pack()


def search_flights():
    rootp.withdraw()  
    root4 = tk.Toplevel()
    root4.title("Welcome, Search Flights")
    root4.geometry("500x400")

    tk.Label(root4, text="Enter Boarding").pack()
    w1 = Combobox(root4, height=5, width=15, values=["New York", "Chicago", "Dallas", "San Francisco"])
    w1.pack()

    tk.Label(root4, text="Select destination").pack()
    w2 = Combobox(root4, height=5, width=15, values=["New York", "Chicago", "Dallas", "San Francisco"])
    w2.pack()

    tk.Label(root4, text="Choose day of travel").pack()
    w3 = Combobox(root4, height=5, width=15, values=["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"])
    w3.pack()

    tk.Label(root4, text="Choose Class").pack()  
    class_entry = Combobox(root4, height=5, width=15, values=["BusinessClass", "Economic"])
    class_entry.pack()

    def search():
        boarding = w1.get()
        destination = w2.get()
        day = w3.get()
        flight_class = class_entry.get() 

        cur = con.cursor()
        if boarding == '' or destination == '' or day == '' or flight_class == '':
            messagebox.showerror("Error", "Cannot leave any field empty")
        else:
            if boarding != destination:
                if flight_class == "Economic":
                    cur.execute("SELECT * FROM economict WHERE boarding=%s AND destination=%s AND day=%s", (boarding, destination, day))
                    flights = cur.fetchall()
                    if flights:
                        messagebox.showinfo("Flights Available", "\n".join(str(flight) for flight in flights))
                    else:
                        messagebox.showinfo("No Flights", "No flights available for the selected criteria.")
                elif flight_class == "BusinessClass":
                    cur.execute("SELECT * FROM businesst WHERE boarding=%s AND destination=%s AND day=%s", (boarding, destination, day))
                    flights = cur.fetchall()
                    if flights:
                        messagebox.showinfo("Flights Available", "\n".join(str(flight) for flight in flights))
                    else:
                        messagebox.showinfo("No Flights", "No flights available for the selected criteria.")
                else:
                    messagebox.showerror("Error", "Invalid class selected")
            else:
                messagebox.showerror("Oops", "Boarding and destination can't be the same")
        return_to_home(root4) 
    
    tk.Button(root4, text="Search", command=search, bg="green", fg="white",font=("Helvetica", 12), padx=10).pack()

    root4.mainloop()

def book_flight():
    rootp.withdraw()  
    root = tk.Toplevel()
    root.title('Flight Search And Booking')
    root.geometry("500x400")

    tk.Label(root, text="Enter Boarding").pack()
    w = Combobox(root, height=5, width=15, values=["Dallas", "Chicago", "New York", "San Francisco"])
    w.pack()

    tk.Label(root, text='Enter Destination').pack()
    w1 = Combobox(root, height=5, width=15, values=["Dallas", "Chicago", "New York", "San Francisco"])
    w1.pack()

    tk.Label(root, text='Enter last 4 digits of your Citizenship Number').pack()
    e = tk.Entry(root, width=20)
    e.pack()

    tk.Label(root, text='Choose Class').pack()
    w2 = Combobox(root, text='BusinessClass', height=5, width=15, values=["BusinessClass", "Economic"])
    w2.pack()
    tk.Label(root, text="Choose day of travel").pack()
    w3 = Combobox(root, text="choose day", height=5, width=15, values=["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"])
    w3.pack()

    tk.Label(root, text="Choose time of your flight").pack()
    w4 = Combobox(root, height=5, width=15, values=["1:00 AM", "7:00 AM", "1:00 PM", "4:00 PM", "9:00 PM"])
    w4.pack()

    def insert():
        boarding = w.get()
        destination = w1.get()
        adno = e.get()
        flight_class = w2.get()
        day = w3.get()
        time = w4.get()
        
        if boarding == '' or destination == '' or adno == '' or flight_class == '' or day == '' or time == '':
            messagebox.showerror("Oops", "You can't leave any field empty")
        else:
            cur = con.cursor()
            if flight_class == 'Economic':
                cur.execute("SELECT * FROM economic WHERE boarding=%s AND destination=%s", (boarding, destination))
                existing_record = cur.fetchone()
                if existing_record:
                    messagebox.showerror("Error", "This route already exists in the Economic class")
                else:
                    cur.execute("INSERT INTO economic (boarding, destination, adno, day, time) VALUES (%s, %s, %s, %s, %s)", (boarding, destination, adno, day, time))
                    messagebox.showinfo("Congratulations", "Your seat has been reserved")
                    con.commit()
                    cur.execute("SELECT * FROM economic WHERE adno=%s", (adno,))
                    messagebox.showinfo("Records", cur.fetchall())
            elif flight_class == 'BusinessClass':
                cur.execute("SELECT * FROM business WHERE boarding=%s AND destination=%s", (boarding, destination))
                existing_record = cur.fetchone()
                if existing_record:
                    messagebox.showerror("Error", "This route already exists in the Business class")
                else:
                    cur.execute("INSERT INTO business (boarding, destination, adno, day, time) VALUES (%s, %s, %s, %s, %s)", (boarding, destination, adno, day, time))
                    messagebox.showinfo("Congratulations", "Your seat has been reserved")
                    con.commit()
                    cur.execute("SELECT * FROM business WHERE adno=%s", (adno,))
                    messagebox.showinfo("Records", cur.fetchall())
            else:
                messagebox.showerror("Error", "Invalid class selected")
            cur.execute("SELECT * FROM vw_economic_reservations")
            economic_records = cur.fetchall()
            cur.execute("SELECT * FROM vw_business_reservations")
            business_records = cur.fetchall()
            messagebox.showinfo("Updated Records", f"Economic Records: {economic_records}\nBusiness Records: {business_records}")
        return_to_home(root) 
    tk.Button(root, text="Insert", command=insert, bg="blue", fg="white",font=("Helvetica", 12), padx=20).pack()

rootp = tk.Tk()
rootp.title("Simple Airline Booking System")
rootp.geometry("800x400")
tk.Label(rootp, text="Welcome to Airline Booking System", font=("Helvetica", 26)).pack()
tk.Button(rootp, text="Cancel Booking", command=cancel_reservation, bg="red", fg="white",font=("Helvetica", 15), padx=20).pack(pady=25)
tk.Button(rootp, text="Search Flight", command=search_flights, bg="green", fg="white",font=("Helvetica", 15), padx=20).pack(pady=25)
tk.Button(rootp, text="Book Flight", command=book_flight, bg="blue", fg="white",font=("Helvetica", 15), padx=20).pack(pady=25)

rootp.mainloop()


In [6]:
import tkinter as tk
from tkinter import messagebox
from tkinter.ttk import Combobox
import mysql.connector

con = mysql.connector.connect(
    host="localhost",
    user="root",
    password="mysql",
    database="airline_reservation"
)

def login():
    def authenticate():
        username = e_username.get()
        password = e_password.get()
        
        cur = con.cursor()
        cur.execute("SELECT * FROM users WHERE username=%s AND password=%s", (username, password))
        user = cur.fetchone()
        if user:
            messagebox.showinfo("Login Successful", "Welcome, " + username)
            root_login.destroy()
            home_page()
        else:
            messagebox.showerror("Login Failed", "Invalid username or password")
    
    root_login = tk.Toplevel()
    root_login.title("Login")
    root_login.geometry("300x150")
    
    tk.Label(root_login, text="Username:").pack()
    e_username = tk.Entry(root_login)
    e_username.pack()
    
    tk.Label(root_login, text="Password:").pack()
    e_password = tk.Entry(root_login, show="*")
    e_password.pack()
    
    tk.Button(root_login, text="Login", command=authenticate,bg="orange").pack()

def register():
    def create_account():
        username = e_username.get()
        password = e_password.get()
        phone = e_phone.get()
        
        cur = con.cursor()
        cur.execute("INSERT INTO users (username, password, phone) VALUES (%s, %s, %s)", (username, password, phone))
        con.commit()
        messagebox.showinfo("Registration Successful", "Account created successfully")
        root_register.destroy()
    
    root_register = tk.Toplevel()
    root_register.title("Register")
    root_register.geometry("600x400")
    
    tk.Label(root_register, text="Username:").pack()
    e_username = tk.Entry(root_register)
    e_username.pack()
    
    tk.Label(root_register, text="Password:").pack()
    e_password = tk.Entry(root_register, show="*")
    e_password.pack()
    
    tk.Label(root_register, text="Phone:").pack()
    e_phone = tk.Entry(root_register)
    e_phone.pack()
    
    tk.Button(root_register, text="Register", command=create_account,bg="yellow").pack()

def home_page():
    root_home = tk.Tk()
    root_home.title("Airline Booking System")
    root_home.geometry("400x200")
    
    tk.Label(root_home, text="Welcome to Airline Booking System", font=("Helvetica", 16)).pack()
    tk.Button(root_home, text="Book Flight", command=book_flight,bg="blue").pack(pady=5)
    tk.Button(root_home, text="Search Flight", command=search_flights,bg="red").pack(pady=5)
    tk.Button(root_home, text="Cancel Booking", command=cancel_reservation,bg="green").pack(pady=5)
    root_home.mainloop()

def return_to_home(root_window):
    root_window.destroy()
    rootp.deiconify()

def cancel_reservation():
    rootp.withdraw() 
    root2 = tk.Toplevel()
    root2.title("Welcome, Customer To our Cancellation System")
    root2.geometry("800x600")

    tk.Label(root2, text="Enter last 4 digits of your Citizenship Number").pack()
    e1 = tk.Entry(root2)
    e1.pack()

    tk.Label(root2, text="Choose your class").pack()
    w2 = Combobox(root2, height=15, width=15, values=["BusinessClass", "Economic"])
    w2.pack()

    tk.Label(root2, text="Select boarding").pack()
    w3 = Combobox(root2, height=15, width=15, values=["New York", "Chicago", "Dallas", "San Francisco"])
    w3.pack()

    def cancel():
        adno = e1.get()
        b = w2.get()
        c = w3.get()
        
        cur = con.cursor()
        if adno == '' or b == '' or c == '':
            messagebox.showerror("Oops", "You can't leave any field empty")
        else:
            if b.lower() == "economic":
                cur.execute("SELECT * FROM economic WHERE adno=%s AND boarding=%s", (adno, c))
            else:
                cur.execute("SELECT * FROM business WHERE adno=%s AND boarding=%s", (adno, c))
                
            records = cur.fetchall()
            if not records:
                messagebox.showinfo("No Seats Found", "No seats found for cancellation.")
            else:
                if b.lower() == "economic":
                    cur.execute("DELETE FROM economic WHERE adno=%s AND boarding=%s", (adno, c))
                else:
                    cur.execute("DELETE FROM business WHERE adno=%s AND boarding=%s", (adno, c))
                    
                con.commit()
                messagebox.showinfo("Reservation Cancelled", "Your reservation has been cancelled.")
                
        return_to_home(root2)  
    
    tk.Button(root2, text="Cancel Reservation", command=cancel, bg="red", fg="white",padx=20).pack()

def search_flights():
    rootp.withdraw()  
    root4 = tk.Toplevel()
    root4.title("Welcome, Search Flights")
    root4.geometry("500x400")

    tk.Label(root4, text="Enter Boarding").pack()
    w1 = Combobox(root4, height=5, width=15, values=["New York", "Chicago", "Dallas", "San Francisco"])
    w1.pack()

    tk.Label(root4, text="Select destination").pack()
    w2 = Combobox(root4, height=5, width=15, values=["New York", "Chicago", "Dallas", "San Francisco"])
    w2.pack()

    tk.Label(root4, text="Choose day of travel").pack()
    w3 = Combobox(root4, height=5, width=15, values=["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"])
    w3.pack()

    tk.Label(root4, text="Choose Class").pack()  
    class_entry = Combobox(root4, height=5, width=15, values=["BusinessClass", "Economic"])
    class_entry.pack()

    def search():
        boarding = w1.get()
        destination = w2.get()
        day = w3.get()
        flight_class = class_entry.get() 

        cur = con.cursor()
        if boarding == '' or destination == '' or day == '' or flight_class == '':
            messagebox.showerror("Error", "Cannot leave any field empty")
        else:
            if boarding != destination:
                if flight_class == "Economic":
                    cur.execute("SELECT * FROM economict WHERE boarding=%s AND destination=%s AND day=%s", (boarding, destination, day))
                    flights = cur.fetchall()
                    if flights:
                        messagebox.showinfo("Flights Available", "\n".join(str(flight) for flight in flights))
                    else:
                        messagebox.showinfo("No Flights", "No flights available for the selected criteria.")
                elif flight_class == "BusinessClass":
                    cur.execute("SELECT * FROM businesst WHERE boarding=%s AND destination=%s AND day=%s", (boarding, destination, day))
                    flights = cur.fetchall()
                    if flights:
                        messagebox.showinfo("Flights Available", "\n".join(str(flight) for flight in flights))
                    else:
                        messagebox.showinfo("No Flights", "No flights available for the selected criteria.")
                else:
                    messagebox.showerror("Error", "Invalid class selected")
            else:
                messagebox.showerror("Oops", "Boarding and destination can't be the same")
        return_to_home(root4) 
    
    tk.Button(root4, text="Search", command=search, bg="green", fg="white").pack()

def book_flight():
    rootp.withdraw()  
    root = tk.Toplevel()
    root.title('Flight Search And Booking')
    root.geometry("500x400")

    tk.Label(root, text="Enter Boarding").pack()
    w = Combobox(root, height=5, width=15, values=["Dallas", "Chicago", "New York", "San Francisco"])
    w.pack()

    tk.Label(root, text='Enter Destination').pack()
    w1 = Combobox(root, height=5, width=15, values=["Dallas", "Chicago", "New York", "San Francisco"])
    w1.pack()

    tk.Label(root, text='Enter last 4 digits of your Citizenship Number').pack()
    e = tk.Entry(root, width=20)
    e.pack()

    tk.Label(root, text='Choose Class').pack()
    w2 = Combobox(root, text='BusinessClass', height=5, width=15, values=["BusinessClass", "Economic"])
    w2.pack()
    tk.Label(root, text="Choose day of travel").pack()
    w3 = Combobox(root, text="choose day", height=5, width=15, values=["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"])
    w3.pack()

    tk.Label(root, text="Choose time of your flight").pack()
    w4 = Combobox(root, height=5, width=15, values=["1:00 AM", "7:00 AM", "1:00 PM", "4:00 PM", "9:00 PM"])
    w4.pack()

    def insert():
        boarding = w.get()
        destination = w1.get()
        adno = e.get()
        flight_class = w2.get()
        day = w3.get()
        time = w4.get()
        
        if boarding == '' or destination == '' or adno == '' or flight_class == '' or day == '' or time == '':
            messagebox.showerror("Oops", "You can't leave any field empty")
        else:
            cur = con.cursor()
            if flight_class == 'Economic':
                cur.execute("SELECT * FROM economic WHERE boarding=%s AND destination=%s", (boarding, destination))
                existing_record = cur.fetchone()
                if existing_record:
                    messagebox.showerror("Error", "This route already exists in the Economic class")
                else:
                    cur.execute("INSERT INTO economic (boarding, destination, adno, day, time) VALUES (%s, %s, %s, %s, %s)", (boarding, destination, adno, day, time))
                    messagebox.showinfo("Congratulations", "Your seat has been reserved")
                    con.commit()
                    cur.execute("SELECT * FROM economic WHERE adno=%s", (adno,))
                    messagebox.showinfo("Records", cur.fetchall())
            elif flight_class == 'BusinessClass':
                cur.execute("SELECT * FROM business WHERE boarding=%s AND destination=%s", (boarding, destination))
                existing_record = cur.fetchone()
                if existing_record:
                    messagebox.showerror("Error", "This route already exists in the Business class")
                else:
                    cur.execute("INSERT INTO business (boarding, destination, adno, day, time) VALUES (%s, %s, %s, %s, %s)", (boarding, destination, adno, day, time))
                    messagebox.showinfo("Congratulations", "Your seat has been reserved")
                    con.commit()
                    cur.execute("SELECT * FROM business WHERE adno=%s", (adno,))
                    messagebox.showinfo("Records", cur.fetchall())
            else:
                messagebox.showerror("Error", "Invalid class selected")
            cur.execute("SELECT * FROM economic")
            economic_records = cur.fetchall()
            cur.execute("SELECT * FROM business")
            business_records = cur.fetchall()
            messagebox.showinfo("Updated Records", f"Economic Records: {economic_records}\nBusiness Records: {business_records}")
        return_to_home(root) 
    tk.Button(root, text="Insert", command=insert, bg="blue", fg="white").pack()

rootp = tk.Tk()
rootp.title("Simple Airline Booking System")
rootp.geometry("400x200")
tk.Label(rootp, text="Welcome to Airline Booking System", font=("Helvetica", 16)).pack()
tk.Button(rootp, text="Login", command=login).pack(pady=5)
tk.Button(rootp, text="Register", command=register).pack(pady=5)
rootp.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\bhave\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\bhave\AppData\Local\Temp/ipykernel_10260/469015454.py", line 24, in authenticate
    home_page()
  File "C:\Users\bhave\AppData\Local\Temp/ipykernel_10260/469015454.py", line 81, in home_page
    root_home.mainloop()
  File "C:\Users\bhave\anaconda3\lib\tkinter\__init__.py", line 1429, in mainloop
    self.tk.mainloop(n)
KeyboardInterrupt


KeyboardInterrupt: 